# World Crop Production in 2018
### Data sources
- `world_food_production.csv`: 2018 world crop production data provided by CFG, original source from [UN FAO](http://www.fao.org/faostat/en/#data/QC)
- `country-codes.csv`: [ISO 3166-1 alpha-3](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3) country codes from [DataHub](https://datahub.io/core/country-codes#resource-country-codes)

### Sections
- **[Part 1](#Part-1:-Data-import-and-cleaning)**: Data import and cleaning
- **[Part 2](#Part-2:-Adding-country-codes)**: Adding country codes
- **[Part 3](#Part-3:-Plotting-the-data-using-Plotly)**: Plotting the data using **[Plotly](https://plotly.com/python/)**
    - **[3a)](#3a.-World-production-&-top-20-producers-by-crop-category) World production & top 20 producers by crop category** - Choropleth maps and bar charts
    - **[3b)](#3b.-Top-countries-by-total-crop-production) Top countries by total crop production** - Horizontal bar chart
    - **[3c)](#3c.-Visualising-data-along-continent-%3E-country-hierarchy) Visualising data along continent > country hierarchy** - Interactive treemap
    - **[3d)](#3d.-Crop-production-in-user-selected-country) Crop production in user-selected country** - Linked pie chart and bar chart, with interactive exploration using **[ipywidgets](https://plotly.com/python/figurewidget-app/)**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from random import choice, randint

In [3]:
import plotly.graph_objects as go 
import plotly.express as px
from ipywidgets import widgets

In [4]:
import cufflinks as cf

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [6]:
init_notebook_mode(connected=True)

In [7]:
cf.go_offline()

# Part 1: Data import and cleaning

In [8]:
data = pd.read_csv('world_food_production_2018.csv')
data.columns

Index(['Country', 'Area', 'Citrus_fruit', 'Eggs', 'Treenuts', 'Pulses',
       'Coarse_grain', 'Meat', 'Roots_and_tubers', 'Vegetables', 'Milk',
       'Fruit', 'Cereals'],
      dtype='object')

In [9]:
# calculate total production
data['Total'] = data.sum(axis=1, numeric_only=True)
data

,Country,Area,Citrus_fruit,Eggs,Treenuts,Pulses,Coarse_grain,Meat,Roots_and_tubers,Vegetables,Milk,Fruit,Cereals,Total
0,Afghanistan,Asia,7510,22373,49399,61778,175091,330636,615684,742001,2128119,2174976,4140568,10448135
1,Albania,Europe,45500,52333,13196,27486,437902,96034,254543,880641,1144365,787834,678196,4418030
2,Algeria,Africa,1478053,314043,57213,137710,2083348,806392,4653322,6910245,3064927,6703946,6064881,32274080
3,American Samoa,Oceania,32,36,0,0,0,344,11522,1069,33,1547,0,14583
4,Angola,Africa,311450,5100,2170,314932,2385484,262490,10740975,815032,217567,4440586,2444745,21940531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Wallis and Futuna Islands,Oceania,48,45,0,0,0,381,4656,625,31,11810,0,17596
211,Western Sahara,Africa,0,0,0,0,1857,2518,0,0,6230,0,1857,12462
212,Yemen,Middle east,153916,55632,168,84865,372814,424876,249244,440216,346581,1092839,478317,3699468
213,Zambia,Africa,3987,52597,0,31752,2449083,321329,1267786,436239,499164,115091,2606609,7783637


In [10]:
# rank production for each crop category
col = list(data.columns)
col = col[2:]

ranks = data.iloc[:,0:2]
for i in range(0, len(col)):
    new_col = col[i]+'_rank'
    ranks[new_col] = data[col[i]].rank(axis=0,method='min', ascending=False)
    
ranks

,Country,Area,Citrus_fruit_rank,Eggs_rank,Treenuts_rank,Pulses_rank,Coarse_grain_rank,Meat_rank,Roots_and_tubers_rank,Vegetables_rank,Milk_rank,Fruit_rank,Cereals_rank,Total_rank
0,Afghanistan,Asia,105.0,115.0,39.0,81.0,119.0,84.0,84.0,78.0,48.0,56.0,61.0,80.0
1,Albania,Europe,78.0,85.0,61.0,98.0,102.0,128.0,118.0,73.0,74.0,87.0,115.0,114.0
2,Algeria,Africa,18.0,37.0,34.0,60.0,62.0,53.0,34.0,19.0,39.0,23.0,47.0,37.0
3,American Samoa,Oceania,145.0,200.0,116.0,174.0,182.0,205.0,167.0,196.0,197.0,195.0,185.0,193.0
4,Angola,Africa,41.0,144.0,90.0,42.0,57.0,95.0,14.0,76.0,121.0,37.0,84.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Wallis and Futuna Islands,Oceania,143.0,199.0,116.0,174.0,182.0,203.0,181.0,200.0,198.0,176.0,185.0,192.0
211,Western Sahara,Africa,147.0,208.0,116.0,174.0,163.0,181.0,205.0,205.0,164.0,208.0,168.0,196.0
212,Yemen,Middle east,54.0,81.0,108.0,70.0,104.0,75.0,119.0,100.0,108.0,81.0,120.0,116.0
213,Zambia,Africa,110.0,84.0,116.0,97.0,55.0,87.0,67.0,101.0,97.0,132.0,82.0,99.0


In [11]:
# convert data to long format
tonnes = data.copy()
tonnes = tonnes.melt(id_vars=['Country', 'Area'], var_name='Type', value_name='Tonnes')

# modify crop category names in 'Type'
tonnes['Type'] = tonnes['Type'].str.replace("_", " ")
tonnes

,Country,Area,Type,Tonnes
0,Afghanistan,Asia,Citrus fruit,7510
1,Albania,Europe,Citrus fruit,45500
2,Algeria,Africa,Citrus fruit,1478053
3,American Samoa,Oceania,Citrus fruit,32
4,Angola,Africa,Citrus fruit,311450
...,...,...,...,...
2575,Wallis and Futuna Islands,Oceania,Total,17596
2576,Western Sahara,Africa,Total,12462
2577,Yemen,Middle east,Total,3699468
2578,Zambia,Africa,Total,7783637


In [12]:
# rank each country by tonnes produced in each crop category
ranks = ranks.melt(id_vars=['Country', 'Area'], var_name='Type', value_name='Rank')
ranks['Type'] = ranks['Type'].str.replace('_rank', '')

# modify crop category names in 'Type'
ranks['Type'] = ranks['Type'].str.replace("_", " ")
ranks

,Country,Area,Type,Rank
0,Afghanistan,Asia,Citrus fruit,105.0
1,Albania,Europe,Citrus fruit,78.0
2,Algeria,Africa,Citrus fruit,18.0
3,American Samoa,Oceania,Citrus fruit,145.0
4,Angola,Africa,Citrus fruit,41.0
...,...,...,...,...
2575,Wallis and Futuna Islands,Oceania,Total,192.0
2576,Western Sahara,Africa,Total,196.0
2577,Yemen,Middle east,Total,116.0
2578,Zambia,Africa,Total,99.0


In [13]:
# merge tonnes and rank data
data_long = tonnes.merge(ranks, on=['Country', 'Area', 'Type']).copy()
data_long.to_csv('world_food_production_2018_long.csv', index=False)
data_long

,Country,Area,Type,Tonnes,Rank
0,Afghanistan,Asia,Citrus fruit,7510,105.0
1,Albania,Europe,Citrus fruit,45500,78.0
2,Algeria,Africa,Citrus fruit,1478053,18.0
3,American Samoa,Oceania,Citrus fruit,32,145.0
4,Angola,Africa,Citrus fruit,311450,41.0
...,...,...,...,...,...
2575,Wallis and Futuna Islands,Oceania,Total,17596,192.0
2576,Western Sahara,Africa,Total,12462,196.0
2577,Yemen,Middle east,Total,3699468,116.0
2578,Zambia,Africa,Total,7783637,99.0


In [14]:
# DID NOT USE
# extract top 10 for each category
# data_top10 = data_long[(data_long['Rank'] >= 1) & (data_long['Rank'] <= 10)].copy()
# data_top10.to_csv('world_food_production_2018_top10.csv', index=False)

# Part 2: Adding country codes
- Adding ISO 3166-1 alpha-3 country codes obtained from [DataHub](https://datahub.io/core/country-codes#resource-country-codes) to the data
- Allows plotting of data as choropleth map using built-in country geometries in [Plotly](https://plotly.com/python/choropleth-maps/#using-builtin-country-and-state-geometries)

In [15]:
# read in useful columns only
gdf = pd.read_csv('country-codes.csv')[['official_name_en', 'ISO3166-1-Alpha-3', 'CLDR display name']]

# rename columns
gdf.columns = ['nation', 'country_code', 'nation2']
gdf

,nation,country_code,nation2
0,Taiwan,TWN,Taiwan
1,Afghanistan,AFG,Afghanistan
2,Albania,ALB,Albania
3,Algeria,DZA,Algeria
4,American Samoa,ASM,American Samoa
...,...,...,...
245,Western Sahara,ESH,Western Sahara
246,Yemen,YEM,Yemen
247,Zambia,ZMB,Zambia
248,Zimbabwe,ZWE,Zimbabwe


In [16]:
# read data
data = pd.read_csv('world_food_production_2018_long.csv')
data

,Country,Area,Type,Tonnes,Rank
0,Afghanistan,Asia,Citrus fruit,7510,105.0
1,Albania,Europe,Citrus fruit,45500,78.0
2,Algeria,Africa,Citrus fruit,1478053,18.0
3,American Samoa,Oceania,Citrus fruit,32,145.0
4,Angola,Africa,Citrus fruit,311450,41.0
...,...,...,...,...,...
2575,Wallis and Futuna Islands,Oceania,Total,17596,192.0
2576,Western Sahara,Africa,Total,12462,196.0
2577,Yemen,Middle east,Total,3699468,116.0
2578,Zambia,Africa,Total,7783637,99.0


### Identifying countries in original dataset which are not matched in the ISO file
Matching country code to the countries in the original dataset proved to be more challenging than expected due to:
- Symbols in names
- Full official name, common name vs abbreviated names
- Sovereign states and dependencies

In [17]:
# checking for matches between 'Country' and 'nation'
country = pd.merge(data['Country'], gdf['nation'], how='left', left_on='Country', right_on='nation')
country

,Country,nation
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,American Samoa,American Samoa
4,Angola,Angola
...,...,...
2575,Wallis and Futuna Islands,Wallis and Futuna Islands
2576,Western Sahara,Western Sahara
2577,Yemen,Yemen
2578,Zambia,Zambia


In [18]:
# isolate NAs
country_na = country[country['nation'].isna()].copy()
country_na.drop_duplicates('Country', inplace=True)
country_na

,Country,nation
21,Bolivia,NaN
26,Brunei,NaN
43,Cote d'Ivoire,NaN
47,Czech Republic,NaN
58,Eswatini,NaN
60,Falkland Islands,NaN
83,Hong Kong,NaN
88,Iran,NaN
101,Laos,NaN
110,Macao,NaN


In [19]:
# checking if NAs from first match can be matched by 'nation2'
country_na2 = country_na.merge(gdf['nation2'], how='left', left_on='Country', right_on='nation2')
country_na2 = country_na2[country_na2['nation2'].isna()].copy()
country_na2

,Country,nation,nation2
2,Cote d'Ivoire,NaN,NaN
3,Czech Republic,NaN,NaN
4,Eswatini,NaN,NaN
9,Macao,NaN,NaN
10,Mainland China,NaN,NaN
13,Netherlands Antilles,NaN,NaN
15,North Macedonia,NaN,NaN
17,Reunion,NaN,NaN
22,United Kingdom,NaN,NaN
23,USA,NaN,NaN


In [20]:
# extracting unmatched countries for modification
to_mod = list(country_na2['Country'])
mod = {i:to_mod[i] for i in range(0,len(to_mod))}
mod

{0: "Cote d'Ivoire",
 1: 'Czech Republic',
 2: 'Eswatini',
 3: 'Macao',
 4: 'Mainland China',
 5: 'Netherlands Antilles',
 6: 'North Macedonia',
 7: 'Reunion',
 8: 'United Kingdom',
 9: 'USA'}

In [21]:
# extract string in ISO table for Ivory Coast and Reunion due to formatting issues
civ = gdf.iat[gdf[gdf['country_code']=='CIV'].index[0],0]
civ

"Côte d'Ivoire"

In [22]:
reu = gdf.iat[gdf[gdf['country_code']=='REU'].index[0],0]
reu

'Réunion'

In [23]:
# manually changing 'nation' in ISO file to match 'Country' in data
mod_names = {civ: "Cote d'Ivoire",
             "Czechia": 'Czech Republic',
             "Swaziland": 'Eswatini',
             "China, Macao Special Administrative Region": 'Macao',
             "China": 'Mainland China',
             "Bonaire, Sint Eustatius and Saba": 'Netherlands Antilles',
             "The former Yugoslav Republic of Macedonia": 'North Macedonia',
             reu: 'Reunion',
             "United Kingdom of Great Britain and Northern Ireland": 'United Kingdom',
             "United States of America": 'USA'}

gdf.replace({'nation': mod_names}, inplace=True)

gdf.shape

(250, 3)

### Merging country codes after modifying ISO data
Merge is still performed in two steps, first merging with 'nation' then merging the remaining unmatched countries with 'nation2'

In [24]:
# merge data with country code
data_code = data.merge(gdf, how='left', left_on='Country', right_on='nation')

# subset unmerged rows to new variable and drop from original data
data_code_na = data_code[data_code['nation'].isna()].copy()
data_code = data_code.dropna()

data_code_na

,Country,Area,Type,Tonnes,Rank,nation,country_code,nation2
21,Bolivia,South America,Citrus fruit,460663,33.0,NaN,NaN,NaN
26,Brunei,Asia,Citrus fruit,528,131.0,NaN,NaN,NaN
60,Falkland Islands,South Atlantic,Citrus fruit,0,147.0,NaN,NaN,NaN
83,Hong Kong,Asia,Citrus fruit,0,147.0,NaN,NaN,NaN
88,Iran,Middle east,Citrus fruit,3486836,11.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2542,South Korea,Asia,Total,26150114,47.0,NaN,NaN,NaN
2550,Syria,Middle East,Total,13450184,69.0,NaN,NaN,NaN
2553,Tanzania,Africa,Total,41373655,35.0,NaN,NaN,NaN
2573,Venezuela,South America,Total,12467650,72.0,NaN,NaN,NaN


In [25]:
# drop NA columns from merge
data_code_na.drop(list(gdf.columns), axis=1, inplace=True)

# merge remaining unmerged rows from first merge using 'nation2'
data_code_na2 = data_code_na.merge(gdf, how='left', left_on='Country', right_on='nation2')
data_code = data_code.append(data_code_na2)
data_code.reset_index(inplace=True)

data_code

,index,Country,Area,Type,Tonnes,Rank,nation,country_code,nation2
0,0,Afghanistan,Asia,Citrus fruit,7510,105.0,Afghanistan,AFG,Afghanistan
1,1,Albania,Europe,Citrus fruit,45500,78.0,Albania,ALB,Albania
2,2,Algeria,Africa,Citrus fruit,1478053,18.0,Algeria,DZA,Algeria
3,3,American Samoa,Oceania,Citrus fruit,32,145.0,American Samoa,ASM,American Samoa
4,4,Angola,Africa,Citrus fruit,311450,41.0,Angola,AGO,Angola
...,...,...,...,...,...,...,...,...,...
2575,187,South Korea,Asia,Total,26150114,47.0,Republic of Korea,KOR,South Korea
2576,188,Syria,Middle East,Total,13450184,69.0,Syrian Arab Republic,SYR,Syria
2577,189,Tanzania,Africa,Total,41373655,35.0,United Republic of Tanzania,TZA,Tanzania
2578,190,Venezuela,South America,Total,12467650,72.0,Venezuela (Bolivarian Republic of),VEN,Venezuela


In [26]:
# Check country codes added to all countries
data_code[data_code['nation'].isna()]

,index,Country,Area,Type,Tonnes,Rank,nation,country_code,nation2


# Part 3: Plotting the data using [Plotly](https://plotly.com/python/)
## 3a. World production & top 20 producers by crop category
Choropleth maps and bar charts

In [27]:
# setting the crop categories as a variable
categories = list(data_code['Type'].unique())
categories.remove('Total')
categories

['Citrus fruit',
 'Eggs',
 'Treenuts',
 'Pulses',
 'Coarse grain',
 'Meat',
 'Roots and tubers',
 'Vegetables',
 'Milk',
 'Fruit',
 'Cereals']

In [28]:
# list of suitable built-in sequential color scales in Plotly
colour_pal = ['Blues', 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'Greens', 'Oranges', 'Purples',
              'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'Reds', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd']

In [29]:
# plot choropleth map and bar chart for top 20 for each category
for cat in categories:
    sub = data_code[data_code['Type'] == cat]
    
    palette = choice(colour_pal)
    
    graph = dict(type ='choropleth',
            locations = sub['country_code'],
            z = sub['Tonnes'],
            text = sub['Country'],
            colorbar = {'title': 'Tonnes'},
            colorscale=palette)

    layout = dict(title = f'<b>World {cat} production - 2018</b>',
                  margin = dict(l=0,
                      r=0,
                      b=30,
                      t=30,
                      pad=2),
                 geo = dict(showframe=False,
                           projection={'type':'equirectangular'}))

    choromap = go.Figure(data=[graph], layout=layout)

    top20 = sub[(sub['Rank'] >= 1) & (sub['Rank'] <= 20)]
    bar_title = f'Top 20 producers of {cat} - 2018'.capitalize()

    bar = dict(type ='bar',
                x = top20['Country'],
                y = top20['Tonnes'], 
               marker={'color': top20['Tonnes'],
                       'colorscale': palette})

    bar_layout = dict(title = f'<b>{bar_title}</b>',
                     xaxis_title='Country',
                     yaxis_title='Tonnes',
                     xaxis={'categoryorder':'total descending'},
                     template='plotly_dark')

    bar2 = go.Figure(data=[bar], layout=bar_layout)

    iplot(choromap)
    iplot(bar2)

## 3b. Top countries by total crop production
Horizontal bar chart

The number of top countries to show is a variable.

In [30]:
total = data_code[data_code['Type'] == 'Total']

# variable threshold for number of top countries
threshold = 20

# identifying top countries by total
total_top = total[(total['Rank'] >= 1) & (total['Rank'] <= threshold)]
top = list(total_top['Country'])

In [31]:
# extract data for all crop types for the top countries from the original df
data_code['top'] = data_code['Country'].isin(top)
data_top = data_code[data_code['top'] == True]
data_top

,index,Country,Area,Type,Tonnes,Rank,nation,country_code,nation2,top
6,6,Argentina,South America,Citrus fruit,3541671,10.0,Argentina,ARG,Argentina,True
13,13,Bangladesh,Asia,Citrus fruit,160615,52.0,Bangladesh,BGD,Bangladesh,True
23,24,Brazil,South America,Citrus fruit,19273659,2.0,Brazil,BRA,Brazil,True
31,33,Canada,North America,Citrus fruit,0,147.0,Canada,CAN,Canada,True
61,64,France,Europe,Citrus fruit,83849,68.0,France,FRA,France,True
...,...,...,...,...,...,...,...,...,...,...
2547,159,Vietnam,Asia,Fruit,9240654,16.0,Viet Nam,VNM,Vietnam,True
2558,170,Russia,Asia,Cereals,109837875,5.0,Russian Federation,RUS,Russia,True
2563,175,Vietnam,Asia,Cereals,48923673,12.0,Viet Nam,VNM,Vietnam,True
2574,186,Russia,Asia,Total,235731649,5.0,Russian Federation,RUS,Russia,True


In [32]:
# plot horizontal bar graph using for loop for stacked bars by crop category
fig = go.Figure()

for cat in categories:
    sub = data_top[data_top['Type'] == cat]
    
    fig.add_trace(go.Bar(x = sub['Tonnes'],
                         y = sub['Country'],
                         name = cat,
                        orientation = 'h'))

fig.update_layout(title=f'<b>Top {threshold} countries by total crop production - 2018</b>',
                  barmode='stack',
                  xaxis_title='Tonnes',
                  yaxis_title='Country',
                  yaxis={'categoryorder':'total ascending',
                        'tickfont':dict(size=8)},
                 legend_title='<b>Category</b>')
iplot(fig)

## 3c. Visualising data along continent > country hierarchy
- Treemap
- Interactive pathbar allows navigation between world, continent and country level data
- Gives sense of total crop production by continent
    - The colour scale is maxed out at the maximum total crop production at the continent-level

In [33]:
tree_data = data_code[data_code['Type'] != 'Total'].copy()

# set single parent
tree_data['World'] = 'World'

# identify maximum total crop production by continent
max_tonnes = tree_data['Tonnes'].groupby([tree_data['Area']]).sum().max()
max_tonnes

4175072844

In [34]:
tree = px.treemap(tree_data, path=['World', 'Area', 'Country', 'Type'], values='Tonnes',
                  color='Tonnes',
                  color_continuous_scale='haline_r',
                 range_color=[0,max_tonnes],
                 title='<b>World crop production - 2018</b>')

iplot(tree)

## 3d. Crop production in user-selected country
### Interactive exploration using [ipywidgets](https://plotly.com/python/figurewidget-app/)
Allows user to explore crop production in a selected country:
- As proportions: Pie chart
- As values: Bar chart

The two charts are linked. Updating the selected country in one changes the visualisation in the other to that selected country.

### Generate pie chart

In [35]:
default_country = 'Mainland China'

# generate widget for country selection
country_dropdown = widgets.Dropdown(
    options = list(tree_data['Country'].unique()),
    value=default_country,
    description='Country: ',
)

# function to filter data by country
def make_chart(country):
    sub = tree_data[tree_data['Country'] == country]
    labels = sub['Type']
    values = sub['Tonnes']
    return labels, values

# initialise default chart to show
labels, values = make_chart(default_country)

# make pie chart
pie = go.Pie(name='',
             labels=labels, 
             values=values, 
             hole=.3,
             hovertemplate = "<b>%{label}</b>" + "<br>%{percent}</br>" + "%{value} tonnes"
            )

# generate figure widget
pie_w = go.FigureWidget(data=[pie],
                              layout=go.Layout(
                                  title=dict(
                                      text=f'<b>Crop production in {default_country} - 2018</b>'
                                  ))
                              )

# define function to handle widget input and update pie chart
def validate():
    if country_dropdown.value in tree_data['Country'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        labels, values = make_chart(country_dropdown.value)
        
        with pie_w.batch_update():
            pie_w.data[0].labels = labels
            pie_w.data[0].values = values
            pie_w.layout.title = f'<b>Crop production in {country_dropdown.value} - 2018</b>'

### Generate bar chart

In [36]:
# set labels and values to country selected in piechart
labels, values = make_chart(country_dropdown.value)

# generate random colour to use for bar chart
def rand_colour():
    return f'rgb({randint(0,255)}, {randint(0,255)}, {randint(0,255)})'

bar_colour = rand_colour()

# make bar chart
barz = go.Bar(x = labels,
              y = values,
              marker=dict(color=bar_colour))

# generate figure widget
barz = go.FigureWidget(data=[barz],
                              layout=go.Layout(
                                  title=dict(
                                      text=f'<b>Crop production in {country_dropdown.value} - 2018</b>'),
                                  xaxis=dict(categoryorder='total descending')
                                  )
                              )

# define function to handle widget input and update bar chart
def response_bar(change):
    if validate():
        labels, values = make_chart(country_dropdown.value)
        bar_colour = rand_colour()
        
        with barz.batch_update():
            barz.data[0].x = labels
            barz.data[0].y = values
            barz.data[0].marker = dict(color=bar_colour)
            barz.layout.title = f'<b>Crop production in {country_dropdown.value} - 2018</b>'

### Display charts

In [37]:
country_dropdown.observe(response, names="value")

container2 = widgets.HBox([country_dropdown])

widgets.VBox([container2,
              pie_w])

In [38]:
country_dropdown.observe(response_bar, names="value")

widgets.VBox([container2,
              barz])